1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [13]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT

In [14]:
os.getcwd()

'/Users/syonis/project/comparative-gsa/sandbox'

In [15]:
# Define variables
param_path = "./parameters_01.json"

n_samples = 16 # Powers of 2

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples'

os.makedirs(os.path.join(simulation_out_path, 
                         f'raw_simulations_{n_samples}_samples'), 
                         exist_ok=True)

In [16]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [17]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,ven.v_ref,la.v,la.delay,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max
0,324.140512,0.429410,970.557289,2.593650,5.759509,141.391649,5.535208,3.645771,0.302181,0.614511,...,2800,93,150,225,25,150,0,420,25,280
1,213.211374,0.168363,1134.582900,3.092750,11.457833,81.051107,6.337772,5.927350,0.476733,0.449234,...,2800,93,150,225,25,150,0,420,25,280
2,128.027561,0.373035,664.791603,1.807716,9.775524,110.934879,3.831676,4.690519,0.657067,0.244713,...,2800,93,150,225,25,150,0,420,25,280
3,287.120769,0.262023,1448.888220,3.761496,8.444797,167.239972,7.529157,2.905174,0.338195,0.522401,...,2800,93,150,225,25,150,0,420,25,280
4,261.317508,0.322630,1342.111306,4.190009,7.184780,154.969280,3.238999,4.237066,0.583734,0.372392,...,2800,93,150,225,25,150,0,420,25,280
5,161.652984,0.284229,1052.809397,2.240715,10.490833,98.802498,8.447837,2.453679,0.400786,0.650167,...,2800,93,150,225,25,150,0,420,25,280
6,187.100586,0.379042,1665.428894,3.405631,13.271724,126.587614,4.626493,3.330410,0.223835,0.459625,...,2800,93,150,225,25,150,0,420,25,280
7,357.429166,0.190679,720.968269,2.907906,5.615618,187.050197,6.917702,5.610030,0.537462,0.294427,...,2800,93,150,225,25,150,0,420,25,280
8,344.802852,0.354682,1594.916363,1.992060,12.260191,176.213767,3.737954,4.435978,0.557119,0.554318,...,2800,93,150,225,25,150,0,420,25,280
9,199.779871,0.243073,791.490741,4.350779,6.556907,103.246173,8.935367,2.134089,0.437717,0.269595,...,2800,93,150,225,25,150,0,420,25,280


In [18]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [19]:
br.map_vessel_volume()

In [20]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,ao.v,art.v,ven.v
0,324.140512,0.429410,970.557289,2.593650,5.759509,141.391649,5.535208,3.645771,0.302181,0.614511,...,103.265426,11.473936,68.843618,0,192.762129,11.473936,128.508086,11.183394,67.548099,3682.353190
1,213.211374,0.168363,1134.582900,3.092750,11.457833,81.051107,6.337772,5.927350,0.476733,0.449234,...,264.970053,29.441117,176.646702,0,494.610765,29.441117,329.740510,12.893425,236.845967,6206.977557
2,128.027561,0.373035,664.791603,1.807716,9.775524,110.934879,3.831676,4.690519,0.657067,0.244713,...,173.562758,19.284751,115.708506,0,323.983815,19.284751,215.989210,14.695447,71.213695,4370.200691
3,287.120769,0.262023,1448.888220,3.761496,8.444797,167.239972,7.529157,2.905174,0.338195,0.522401,...,132.781531,14.753503,88.521021,0,247.858858,14.753503,165.239239,8.718682,125.161948,5564.827928
4,261.317508,0.322630,1342.111306,4.190009,7.184780,154.969280,3.238999,4.237066,0.583734,0.372392,...,146.342358,16.260262,97.561572,0,273.172401,16.260262,182.114934,10.757573,139.708922,5167.193956
5,161.652984,0.284229,1052.809397,2.240715,10.490833,98.802498,8.447837,2.453679,0.400786,0.650167,...,212.449989,23.605554,141.633326,0,396.573313,23.605554,264.382209,13.534576,106.699518,4704.827721
6,187.100586,0.379042,1665.428894,3.405631,13.271724,126.587614,4.626493,3.330410,0.223835,0.459625,...,226.146168,25.127352,150.764112,0,422.139514,25.127352,281.426342,17.698409,159.017196,5910.682839
7,357.429166,0.190679,720.968269,2.907906,5.615618,187.050197,6.917702,5.610030,0.537462,0.294427,...,89.740137,9.971126,59.826758,0,167.514922,9.971126,111.676615,4.153311,63.339163,4074.273614
8,344.802852,0.354682,1594.916363,1.992060,12.260191,176.213767,3.737954,4.435978,0.557119,0.554318,...,183.766483,20.418498,122.510989,0,343.030768,20.418498,228.687179,10.849859,60.937888,5390.448288
9,199.779871,0.243073,791.490741,4.350779,6.556907,103.246173,8.935367,2.134089,0.437717,0.269595,...,123.195791,13.688421,82.130528,0,229.965477,13.688421,153.310318,9.935395,177.834472,4220.101290


In [21]:
# Save the samples to a CSV file

br.samples.to_csv(os.path.join(simulation_out_path,
                               f'input_samples_{n_samples}.csv'),
                                 index=False)

In [22]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [23]:
simulations = br.run_batch(
            n_jobs=10,
            output_path=os.path.join(simulation_out_path, f'raw_simulations_{n_samples}_samples') 
        )

100%|██████████| 16/16 [00:00<00:00, 475.85it/s]

In [24]:
## Setting up the headers of the columns for my simulation summary similar to fay
##headers = [
   # 'v_ao', 'v_art', 'v_ven', 'v_la', 'v_lv', 'p_lv', 'q_av', 'p_ao', 'p_art',
   # 'q_ao', 'p_ven', 'q_art', 'p_la', 'q_ven', 'q_mv', 'T'
#]

summary_rows = []

for i_sim in range(len(simulations)): ## stuck with the length of simulations like fay
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame this was needed because sometimes simulations can return None or other types. or false.
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {} ##the 'dictiionary' to hold the summary for each simulation
    for h in sim_df.columns: # We had sim_df the whole time which means h can be used without the headers list!
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max() ## using the headers defined above to make the new csv file headers eg v_ao_mean, v_ao_max, etc.
    summary_rows.append(row) # this is a list that is being used to collect summary data for our simulations.
    print(f"Simulation {i_sim} done.") # simple way to track progress 

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(os.path.join(simulation_out_path, "simulations_summary.csv"), index=False)  ## the location, nice and organised!
print("Saved as simulation_summary.csv") # not really needed but nice to show the file has been saved! :D

Simulation 0 done.
Simulation 1 done.
Simulation 2 done.
Simulation 3 done.
Simulation 4 done.
Simulation 5 done.
Simulation 6 done.
Simulation 7 done.
Simulation 8 done.
Simulation 9 done.
Simulation 10 done.
Simulation 11 done.
Simulation 12 done.
Simulation 13 done.
Simulation 14 done.
Simulation 15 done.
Saved as simulation_summary.csv
